In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
from sklearn import preprocessing



In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import SGD

In [ ]:
#lets read each file
data = ""
for filename in os.listdir(os.getcwd()+"/Gutenberg/txt"):
    print(filename)
    if filename == "Henry David Thoreau___A Week on the Concord and Merrimack Rivers.txt":
        break;
    with open(os.getcwd()+"/Gutenberg/txt/" + filename, 'r') as file:
        data += file.read().replace('\n', '')
        break

In [ ]:
#wow, thats a lot of data
print(len(data))
data = np.asarray([char for char in data]).reshape(-1,1)

In [ ]:
le = preprocessing.OneHotEncoder()
#lets make a one-hot encoder for the chars
le.fit(data)
data = le.transform(data)

In [ ]:
#lets generate some data!
import random
num_samples = 25000
sample_length = 5

x = []
y = []
i = 0
while len(x) < num_samples:
    
    if len(x) % 1000 == 0:
        print(len(x))
    i = random.randrange(0, data.shape[0]-sample_length-10)
    
    try:
        x.append(data[i:i+sample_length])
        
        y.append(data[i+sample_length+1])
      
    except:
        #I don't really care if it breaks, we have 7 million more samples
   
        continue
x = np.array(x)
y = np.asarray([thing.todense() for thing in y]).reshape(num_samples,-1)

In [ ]:
#reshape x
new_x = np.zeros((num_samples, sample_length, x[0].shape[1]))
for i in range(len(x)):
    new_x[i] = x[i].todense()
x = new_x

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
def build_model(output_size):
    
    main_input = Input(shape=((sample_length,output_size,)))
    lstm_layer = LSTM(512)(main_input)
    d_1 = Dense(128, activation='relu')(lstm_layer)
    output = Dense(output_size, activation='softmax')(d_1)
    
    model = Model(inputs=[main_input], outputs=[output])
    return model

In [ ]:
model = build_model(len(le.categories_[0]))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(new_x,y,epochs=100, batch_size=32)

In [ ]:
def generate_text(model, length, sample_length, inp):
    output = inp
    for i in range(length):
        new_inp = np.asarray([mat.todense() for mat in le.transform(np.asarray([char for char in output[-sample_length:]]).reshape(-1,1))]).reshape(1,sample_length,-1)
        encoded = model.predict(new_inp)
        next_char = le.inverse_transform(encoded)
        output = output + next_char[0][0]
    return output


In [ ]:
generate_text(model,300,  sample_length, "once upon a time i decided")

In [ ]:
model.save("rnn_model")

In [ ]:
rnn_model = keras.models.load_model("rnn_model")


In [ ]:
generate_text(rnn_model, 300, sample_length, "Input your sample text here")